train, validation은 일단 나눔. 그리고 cbam적용하는 것이 목표임.

In [1]:
import pandas as pd
from pathlib import Path
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from torch.utils.data import Subset

In [2]:
Catholic_file = 'aug_train_v2.2.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,7
1,0002-2.wav,non-wheezing,0,1,7
2,0002-3.wav,non-wheezing,0,1,7
3,0002-4.wav,non-wheezing,0,1,7
4,0003-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,9
1606,wn_0588-1.wav,wheezing,1,0,1
1607,wn_0599-3.wav,wheezing,1,1,6
1608,wn_0602-2.wav,wheezing,1,1,8


In [3]:
df = df.sample(frac=1)

In [4]:
df

,filename,category,class,sex,old
1105,rs_0581-4.wav,wheezing,1,1,7
21,0010-1.wav,non-wheezing,0,1,3
689,cs_0615-4.wav,wheezing,1,1,11
807,minus_0042-4.wav,non-wheezing,0,1,3
497,cs_0015-2.wav,non-wheezing,0,1,2
...,...,...,...,...,...
387,cp_0261-2.wav,wheezing,1,0,7
450,cp_0108-2.wav,wheezing,1,0,8
487,cs_0011-4.wav,non-wheezing,0,1,0
398,cp_0534-4.wav,wheezing,1,0,9


In [5]:
mean = df.old.mean()
std = df.old.std()
df.old = (df.old - mean) / std

In [6]:
df

,filename,category,class,sex,old
1105,rs_0581-4.wav,wheezing,1,1,0.905217
21,0010-1.wav,non-wheezing,0,1,-0.330391
689,cs_0615-4.wav,wheezing,1,1,2.140826
807,minus_0042-4.wav,non-wheezing,0,1,-0.330391
497,cs_0015-2.wav,non-wheezing,0,1,-0.639293
...,...,...,...,...,...
387,cp_0261-2.wav,wheezing,1,0,0.905217
450,cp_0108-2.wav,wheezing,1,0,1.214119
487,cs_0011-4.wav,non-wheezing,0,1,-1.257097
398,cp_0534-4.wav,wheezing,1,0,1.523021


In [7]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class', 'sex', 'old']]
df.head()

,relative_path,class,sex,old
1105,/rs_0581-4.wav,1,1,0.905217
21,/0010-1.wav,0,1,-0.330391
689,/cs_0615-4.wav,1,1,2.140826
807,/minus_0042-4.wav,0,1,-0.330391
497,/cs_0015-2.wav,0,1,-0.639293


In [8]:
data_path = 'aug_train_v2.2'

In [9]:
class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [10]:
class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 22050
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df.loc[idx, 'old']
    y = self.df.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    
    
    return [aug_sgram, tabular], class_id

In [11]:
from skorch import NeuralNetClassifier

In [12]:
ds = breathDS(df, data_path)

In [13]:
'''num_items = len(brds)
print(len(brds))
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(brds, [num_train, num_val])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=True)'''

'num_items = len(brds)\nprint(len(brds))\nnum_train = round(num_items * 0.8)\nnum_val = num_items - num_train\ntrain_ds, val_ds = random_split(brds, [num_train, num_val])\n\ntrain_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)\nval_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=True)'

In [14]:
#train_dl = DataLoader(brds, batch_size=16, shuffle=True)

In [15]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [16]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        #self.fc1 = nn.Linear(2, 8)
        #self.fc = nn.Linear((512 * block.expansion)+8, num_classes)
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc_tab = nn.Linear(2, 8)
        self.fc_tab2 = nn.Linear(8, 16)
        self.fc = nn.Linear((512 * block.expansion)+16, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

        #self.fc1 = nn.Linear(2, 8)
        #self.fc2 = nn.Linear(8, 16)
        #self.fc = nn.Linear((512 * block.expansion)+16, num_classes)
        #self.fc = nn.Linear((512 * block.expansion), 128)
        #self.fc_ = nn.Linear(160, num_classes)

    def forward(self, x_):
        x = x_[0]
        x2 = x_[1]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        #print(x.size())
        #x2 = self.fc1(x2)
        #x2 = self.fc2(x2)
        #print(x2.size())
        #x_ = torch.cat((x1, x2), 1)
        #x = self.fc(x)
        x2 = self.fc_tab(x2)
        x2 = self.fc_tab2(x2)
        #x2 = self.fc2(x2)
        #print(x.size())
        #print(x2.size())
        x = torch.cat((x, x2), 1)
        x = self.fc(x)
        return x

In [17]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from skorch.helper import SliceDataset

Model1 = ResNet(BasicBlock, [2, 2, 2, 2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

params = {
    'lr': [0.001, 0.01],
    'max_epochs': [40, 60, 80],
    'batch_size': [16, 32]
}

net = NeuralNetClassifier(
    Model1,
    optimizer=torch.optim.Adam,
    criterion=nn.CrossEntropyLoss,
    max_epochs=80,
    lr=0.001,
    batch_size=16,
    iterator_train__shuffle=True,
    verbose=1,
    train_split=None
)

In [18]:
x_ = SliceDataset(ds, idx=0)
outputs = SliceDataset(ds, idx=1)

In [19]:
from skorch.helper import SliceDataset

#gs = GridSearchCV(net, params, refit=False, cv=5, scoring='accuracy', error_score="raise", verbose=3, n_jobs=-1)
gg = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
gs = GridSearchCV(net, params, refit=False, cv=gg, scoring='accuracy', error_score="raise", verbose=3)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [21]:
gs.fit(x_, outputs)

pd.DataFrame(gs.cv_results_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/nextgen/anaconda3/envs/kbs_3060/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  epoch    train_loss      dur
-------  ------------  -------
      1        0.4328  35.6904
      2        0.3857  32.8897
      3        0.3539  35.0175
      4        0.3746  31.5391
      5        0.3781  28.4739
      6        0.3486  28.2469
      7        0.3611  28.9684
      8        0.3517  29.8711
      9        0.3376  28.9848
     10        0.3273  28.9279
     11        0.3060  28.4486
     12        0.3130  29.8069
     13        0.2961  29.2132
     14        0.2895  28.2574
     15        0.2694  28.1237
     16        0.2657  27.5992
     17        0.2496  27.6085
     18        0.2282  27.5776
     19        0.2324  27.5819
     20        0.2129  27.5706
     21        0.2027  27.7277
     22        0.1878  27.7005
     23        0.1962  27.6702
     24        0.1612  28.7530
     25        0.1768  31.1267
     26        0.1826  31.8287
     27        0.1431  29.5144
     28        0.1257  29.3228
     29        0.1309  30.5935
     30        0.1224  30.1069
     31 

      3        0.3857  27.8673
      4        0.3650  28.5805
      5        0.3570  28.0511
      6        0.3628  28.9034
      7        0.3505  27.8441
      8        0.3367  29.2521
      9        0.3341  28.2375
     10        0.3107  28.7352
     11        0.3185  29.3530
     12        0.3025  29.4627
     13        0.2784  28.6358
     14        0.2861  29.1840
     15        0.2647  29.1849
     16        0.2524  28.8177
     17        0.2390  28.2498
     18        0.2221  30.0158
     19        0.2232  28.6442
     20        0.2152  28.8832
     21        0.1836  27.8555
     22        0.1773  27.7898
     23        0.1632  27.6680
     24        0.1735  28.2876
     25        0.1612  28.5192
     26        0.1303  29.4371
     27        0.1416  28.6511
     28        0.1470  28.0895
     29        0.1370  28.1024
     30        0.1015  28.6730
     31        0.0963  27.9200
     32        0.1103  28.0997
     33        0.0977  29.0495
     34        0.1185  27.8489
     35 

     42        0.0611  27.6301
     43        0.0838  27.6059
     44        0.0782  27.5333
     45        0.0998  27.5282
     46        0.0709  27.5425
     47        0.0547  27.5040
     48        0.0638  27.5365
     49        0.0439  27.5328
     50        0.0718  27.5612
     51        0.0751  27.5428
     52        0.0482  27.6286
     53        0.0706  27.6766
     54        0.0746  27.7294
     55        0.0661  27.6543
     56        0.0622  27.5804
     57        0.0624  27.6102
     58        0.0398  27.6303
     59        0.0610  27.6390
     60        0.0450  27.5609
[CV 4/5] END batch_size=16, lr=0.001, max_epochs=60;, score=0.898 total time=28.1min
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4376  27.4718
      2        0.3869  27.4764
      3        0.3825  27.4474
      4        0.3818  27.4466
      5        0.3539  27.4587
      6        0.3713  27.4750
      7        0.3503  27.4695
      8        0.3442  27.3855
      9        0

...

      7        0.3459  24.8667
      8        0.3530  24.9118
      9        0.3481  24.3997
     10        0.3467  24.3560
     11        0.3492  24.6350
     12        0.3318  24.6106
     13        0.3407  24.4081
     14        0.3392  24.5098
     15        0.3154  24.5138
     16        0.3257  24.6195
     17        0.3281  24.4913
     18        0.3049  24.5690
     19        0.3270  24.7352
     20        0.3196  24.4630
     21        0.2971  24.4618
     22        0.2877  24.4495
     23        0.3024  24.4088
     24        0.3004  24.4948
     25        0.2837  24.3857
     26        0.2681  24.7412
     27        0.2635  25.0711
     28        0.2689  24.5551
     29        0.2855  24.5505
     30        0.2564  24.5479
     31        0.2943  24.5221
     32        0.2672  24.6259
     33        0.2520  24.5280
     34        0.2887  24.5583
     35        0.2254  24.4172
     36        0.2353  24.4218
     37        0.2277  24.4031
     38        0.1952  24.3792
     39 

     25        0.3245  24.6228
     26        0.2993  24.6717
     27        0.2849  24.6423
     28        0.3052  24.5899
     29        0.2832  24.6578
     30        0.2758  24.7459
     31        0.2746  24.6964
     32        0.2625  24.7088
     33        0.2664  24.8085
     34        0.2593  25.5325
     35        0.2596  24.8789
     36        0.2433  24.6696
     37        0.2542  24.7311
     38        0.2663  24.7865
     39        0.2351  24.6729
     40        0.2155  24.7235
     41        0.1783  24.6050
     42        0.2147  24.6182
     43        0.2060  24.6370
     44        0.2160  24.6841
     45        0.1817  24.8427
     46        0.1981  24.8857
     47        0.1982  24.8529
     48        0.1571  24.9200
     49        0.1884  25.3480
     50        0.1710  24.8832
     51        0.1688  24.8785
     52        0.1826  24.9565
     53        0.1845  25.1085
     54        0.1690  24.9054
     55        0.1519  24.9427
     56        0.1588  24.9265
     57 

     66        0.1140  25.1405
     67        0.1222  25.5777
     68        0.1174  25.4661
     69        0.0808  27.0345
     70        0.0819  25.6902
     71        0.0927  25.6145
     72        0.1342  26.0949
     73        0.0899  26.6405
     74        0.0958  30.2725
     75        0.0942  25.5012
     76        0.0677  25.5670
     77        0.0505  25.5173
     78        0.0555  25.0609
     79        0.0660  25.1993
     80        0.0797  25.3443
[CV 1/5] END batch_size=32, lr=0.01, max_epochs=80;, score=0.870 total time=34.1min
  epoch    train_loss      dur
-------  ------------  -------
      1        0.5564  24.7396
      2        0.4232  24.7993
      3        0.3848  24.7371
      4        0.3960  25.0821
      5        0.3691  24.7458
      6        0.3790  25.0053
      7        0.3579  24.7587
      8        0.3740  25.1358
      9        0.3638  25.2398
     10        0.3426  25.0156
     11        0.3536  24.8592
     12        0.3542  24.8137
     13        0.

     47        0.1659  24.8549
     48        0.2012  24.8047
     49        0.1495  24.8162
     50        0.1593  24.7863
     51        0.1537  24.8109
     52        0.1494  24.8425
     53        0.1547  24.9278
     54        0.1615  25.3637
     55        0.1356  26.3155
     56        0.1824  25.1045
     57        0.1515  25.1084
     58        0.1135  25.0848
     59        0.1389  25.0525
     60        0.1072  25.1725
     61        0.1154  25.1691
     62        0.1049  25.1833
     63        0.0844  25.1939
     64        0.0954  25.6355
     65        0.1186  26.0692
     66        0.1102  25.8550
     67        0.1086  25.3050
     68        0.0711  25.8235
     69        0.0713  25.5491
     70        0.0774  26.0517
     71        0.1203  25.3315
     72        0.1129  25.2547
     73        0.0879  25.2459
     74        0.0808  25.2682
     75        0.0657  25.3155
     76        0.0466  25.4185
     77        0.0735  25.3496
     78        0.0810  25.2268
     79 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_lr,param_max_epochs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1149.204040,28.791707,13.045013,0.370769,16,0.001,40,"{'batch_size': 16, 'lr': 0.001, 'max_epochs': 40}",0.900621,0.937888,0.897516,0.906832,0.850932,0.898758,0.027888,9
1,1681.458399,21.375225,12.913659,0.377485,16,0.001,60,"{'batch_size': 16, 'lr': 0.001, 'max_epochs': 60}",0.906832,0.922360,0.900621,0.897516,0.925466,0.910559,0.011351,4
2,2221.969596,3.378738,12.819789,0.432096,16,0.001,80,"{'batch_size': 16, 'lr': 0.001, 'max_epochs': 80}",0.897516,0.928571,0.934783,0.916149,0.906832,0.916770,0.013665,2
3,1129.347267,4.466733,12.909704,0.391310,16,0.01,40,"{'batch_size': 16, 'lr': 0.01, 'max_epochs': 40}",0.881988,0.928571,0.854037,0.891304,0.847826,0.880745,0.028974,11
4,1707.554254,32.336747,13.257086,0.995476,16,0.01,60,"{'batch_size': 16, 'lr': 0.01, 'max_epochs': 60}",0.891304,0.903727,0.897516,0.906832,0.878882,0.895652,0.009938,10
5,2273.645899,22.463364,12.972561,0.389849,16,0.01,80,"{'batch_size': 16, 'lr': 0.01, 'max_epochs': 80}",0.881988,0.922360,0.913043,0.919255,0.922360,0.911801,0.015290,3
6,990.647284,3.929943,12.811673,0.336206,32,0.001,40,"{'batch_size': 32, 'lr': 0.001, 'max_epochs': 40}",0.903727,0.940994,0.888199,0.891304,0.888199,0.902484,0.020088,7
7,1481.331152,5.487078,12.892663,0.316047,32,0.001,60,"{'batch_size': 32, 'lr': 0.001, 'max_epochs': 60}",0.913043,0.947205,0.891304,0.903727,0.875776,0.906211,0.024008,5
8,1971.130551,4.801546,12.831889,0.371756,32,0.001,80,"{'batch_size': 32, 'lr': 0.001, 'max_epochs': 80}",0.922360,0.922360,0.928571,0.922360,0.897516,0.918634,0.010830,1
9,998.108085,5.765190,12.892324,0.373880,32,0.01,40,"{'batch_size': 32, 'lr': 0.01, 'max_epochs': 40}",0.854037,0.875776,0.847826,0.891304,0.866460,0.867081,0.015516,12
